In [1]:
import plotly.graph_objects as go     #   ใช้สร้างกราฟแบบออบเจ็กต์ (เราจะใช้ go.Choropleth สำหรับแผนที่)
import plotly.io as pio               #   ว้ตั้งค่า renderer ว่าจะแสดงกราฟที่ไหน
import pandas as pd

pio.renderers.default = "notebook_connected"  ## หรือ "vscode" / "jupyterlab" / run on base (Python 3.12.7)

# Backtracking (dict result)
# ถ้ามี "KA" จะ map เป็น "KS" ให้
result = {
    'WA':'R','OR':'G','CA':'Y','NV':'B','AZ':'R','ID':'B','UT':'G','MT':'G','WY':'B','CO':'R',   # R/G/B/Y  R = Orange, G = LightGreen, B = SkyBlue, Y = Yellow
    'NM':'B','ND':'B','SD':'R','NE':'G','KS':'B','OK':'G','TX':'R','MN':'G','IA':'B','MO':'R',
    'AR':'B','LA':'G','WI':'R','IL':'G','IN':'R','MI':'B','OH':'G','KY':'B','TN':'G','MS':'R',
    'AL':'B','GA':'R','FL':'G','SC':'G','NC':'B','VA':'R','WV':'Y','MD':'G','DC':'B','DE':'B',
    'PA':'R','NJ':'G','NY':'B','CT':'G','RI':'B','MA':'R','VT':'G','NH':'B','ME':'R'
}
if 'KA' in result and 'KS' not in result:
    result['KS'] = result.pop('KA')

# เติม Alaska/Hawaii ให้มีสี
result.setdefault('AK', 'B')  # Alaska
result.setdefault('HI', 'Y')  # Hawaii

# กราฟเพื่อนบ้าน
NEIGHBORS = {
    "WA":["ID","OR"], "OR":["WA","ID","NV","CA"], "CA":["OR","NV","AZ"],
    "ID":["WA","OR","NV","UT","WY","MT"], "NV":["OR","ID","UT","AZ","CA"],
    "AZ":["CA","NV","UT","NM"], "UT":["ID","WY","CO","NM","AZ","NV"],
    "MT":["ID","WY","SD","ND"], "WY":["MT","SD","NE","CO","UT","ID"],
    "CO":["WY","NE","KS","OK","NM","AZ","UT"], "NM":["AZ","UT","CO","OK","TX"],
    "ND":["MT","SD","MN"], "SD":["ND","MT","WY","NE","IA","MN"], "NE":["SD","WY","CO","KS","MO","IA"],
    "KS":["NE","CO","OK","MO"], "OK":["CO","NM","TX","AR","MO","KS"], "TX":["NM","OK","AR","LA"],
    "MN":["ND","SD","IA","WI"], "IA":["MN","SD","NE","MO","IL","WI"], "MO":["IA","NE","KS","OK","AR","TN","KY","IL"],
    "AR":["MO","OK","TX","LA","MS","TN"], "LA":["TX","AR","MS"],
    "WI":["MN","IA","IL","MI"], "IL":["WI","IA","MO","KY","IN"], "IN":["IL","KY","OH","MI"],
    "MI":["WI","IN","OH"], "OH":["IN","MI","PA","WV","KY"], "KY":["MO","IL","IN","OH","WV","VA","TN"],
    "TN":["MO","AR","MS","AL","GA","NC","VA","KY"], "MS":["LA","AR","TN","AL"], "AL":["MS","TN","GA","FL"],
    "GA":["AL","TN","NC","SC","FL"], "FL":["AL","GA"], "SC":["GA","NC"], "NC":["SC","GA","TN","VA"],
    "VA":["NC","TN","KY","WV","MD"], "WV":["OH","PA","MD","VA","KY"],
    "PA":["OH","WV","MD","DE","NJ","NY"], "MD":["VA","WV","PA","DC","DE"], "DC":["MD","VA"],
    "DE":["MD","PA","NJ"], "NJ":["DE","PA","NY"], "NY":["PA","NJ","CT","MA","VT"],
    "CT":["NY","MA","RI"], "RI":["CT","MA"], "MA":["NY","VT","NH","RI","CT"],
    "VT":["NY","NH","MA"], "NH":["VT","MA","ME"], "ME":["NH"],
    "AK":[], "HI":[]
}

# ฟังก์ชันแก้สีที่ชนกัน
COLOR_ORDER = ['R','G','B','Y']  # ลองเปลี่ยนสีตามลำดับนี้เวลาต้องแก้

def fix_conflicts(assign: dict) -> dict:
    #  ปรับสีที่ชนกันแบบ greedy: วนเช็กคู่เพื่อนบ้าน ถ้าชนให้หาสีใหม่ที่ไม่ชน
    assign = assign.copy()
    changed = True
    while changed:
        changed = False
        for s, nbrs in NEIGHBORS.items():
            if s not in assign:  # ข้ามรัฐที่ยังไม่มีสี
                continue
            for n in nbrs:
                if n in assign and assign[s] == assign[n]:
                    # หา “สีอื่น” ที่ไม่ชนเพื่อนบ้านของ s
                    for c in COLOR_ORDER:
                        if c != assign[s] and all(assign.get(nn) != c for nn in NEIGHBORS[s]):
                            assign[s] = c
                            changed = True
                            break
                    # ถ้ายังไม่มีสีที่วางได้ในรอบนี้ จะไปแก้ต่อในรอบถัดไป
    return assign

result_fixed = fix_conflicts(result)

#  แปลง ตัวหนังสือสี
def to_idx(v: str) -> int:
    v = str(v).strip().lower()
    return {'r':0,'orange':0,'g':1,'lightgreen':1,'b':2,'skyblue':2,'y':3,'yellow':3}.get(v, 0)   # ฟังก์ชันนี้จะแปลงเป็น เลข 0,1,2,3 เพื่อใช้กับ colorscale แบบ discrete 

# เตรียมข้อมูล Plotly 
df = pd.DataFrame({
    'state': list(result_fixed.keys()),
    'cidx': [to_idx(v) for v in result_fixed.values()]
})

# 4 สี (ส้ม/เขียวอ่อน/ฟ้า/เหลือง) แบบ discrete
#   สีล้วน 4 แถบ ไม่ใช่ไล่เฉด -> จึงกำหนดช่วงของแต่ละสี (เช่น 0.00–0.24 = ส้ม, 0.25–0.49 = เขียวอ่อน ฯลฯ)
colorscale = [
    [0.00, '#FFA500'], [0.24, '#FFA500'],
    [0.25, '#90EE90'], [0.49, '#90EE90'],
    [0.50, '#87CEEB'], [0.74, '#87CEEB'],
    [0.75, '#FFD700'], [1.00, '#FFD700'],
]

#   สร้างชั้นข้อมูล choropleth (แผนที่รัฐ)
fig = go.Figure(data=go.Choropleth(
    locations=df['state'],          #   บอกแต่ละ (รัฐ) ที่จะระบายสีด้วยรหัส 2 ตัว
    z=df['cidx'],                   #   คือค่าที่ใช้ map ลง colorscale (เราเตรียมไว้เป็น 0..3)
    locationmode='USA-states',      #   สั่งให้ plotly ใช้ขอบเขตรัฐตามที่มาพร้อมใน plotly
    colorscale=colorscale,
    showscale=False,                #   ซ่อนแถบ colorbar (เพราะตั้งเป็น discrete)
    marker_line_color='black',      #   ตั้งเส้นขอบรัฐสีดำ เพื่อแบ่งเขต
    marker_line_width=0.6
))

#   ตั้งค่าแผนภาพ
fig.update_layout(
    title="USA Map Color (4 colors)",
    geo=dict(scope='north america', projection_type='albers usa', showlakes=False),   #   scope='usa' โฟกัสแผนที่ USA / projection_type='albers usa' เป็นโปรเจคชัน สัดส่วนสวย และจัด Alaska/Hawaii ให้อยู่ในกรอบ
    margin=dict(l=0, r=0, t=60, b=0),
)
fig.show()
